In [1]:
from code.neuron import *
from code.encodingsource import *
from code.hsgs import *
from code.classical_neuron import *
from code.classical_pso import *
from code.sf import *
simulator = Aer.get_backend('qasm_simulator')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json

from code.experiment_functions import *

In [ ]:
diabetes = pd.read_csv('data/diabetes.csv')

In [ ]:
# normalization 0-1
for i in diabetes.columns:
    diabetes[i] = (diabetes[i] - diabetes[i].min()) / (diabetes[i].max() - diabetes[i].min())

In [ ]:
drop_indices = np.random.choice(diabetes.index, 368, replace=False)
diabetes = diabetes.drop(drop_indices)

In [ ]:
# split in test and train data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(diabetes.drop('Outcome',axis=1), diabetes['Outcome'], test_size=0.2)

In [ ]:
X_train[X_train == 0] = 0.000001
X_test[X_test == 0] = 0.000001

In [ ]:
X_train = X_train.values.tolist()
X_test = X_test.values.tolist()
y_train = y_train.values.tolist()
y_test = y_test.values.tolist()

In [ ]:
y_train = [int(x) for x in y_train]
y_test =  [int(x) for x in y_test]

In [ ]:
"""
with open('results/version4/train_data.json', 'w+') as fp:
    json.dump([X_train, y_train], fp)
    
with open('results/version4/test_data.json', 'w+') as fp:
    json.dump([X_test, y_test], fp)
"""

In [2]:
with open('results/version4/train_data.json') as json_file:
    X_train = json.load(json_file)[0]

with open('results/version4/train_data.json') as json_file:
    y_train = json.load(json_file)[1]
    
with open('results/version4/test_data.json') as json_file:
    X_test = json.load(json_file)[0]

with open('results/version4/test_data.json') as json_file:
    y_test = json.load(json_file)[1]

In [3]:
def float_to_binary(list_, m, n):
    """Convert the float value `x` to a binary string of length `m + n`
    where the first `m` binary digits are the integer part and the last
    'n' binary digits are the fractional part of `x`.
    """
    list_scaled = []
    for number in list_:
        x_scaled = round(number * 2 ** n)
        x_scaled = '{:0{}b}'.format(x_scaled, m + n)
        for i in x_scaled:
            if i == '0':
                list_scaled.append(-1)
            else:
                list_scaled.append(1)
    return list_scaled

X_test_bin = []
for i in X_test:
    X_test_bin.append(float_to_binary(i, 1, 4))
    
X_train_bin = []
for i in X_train:
    X_train_bin.append(float_to_binary(i, 1, 4))

## Test

In [7]:
w = list(np.random.uniform(low=0.0, high=1.0, size=(len(X_test_bin[0]),)))

In [8]:
weights = quantumNeuronFIT(X_test_bin, 
                           y_train,
                           init_weight = w,
                           threshold=0.7,
                           lrParameter=0.1, 
                           n_epochs=20, 
                           trainingBias=False,
                           epoch_results=True,
                           phaseEstrategyOperator='original',
                           trainingApproaches={'hsgs'})

lrParameter:  0.1
threshold:  0.7
trainingBias:  False
phaseEstrategyOperator:  original
best error HSGS training:  33


In [9]:
results = quantumNeuronPREDICT(X_test, 
                                   y_test,
                                   threshold=0.7,
                                   weightVectorsPhaseEncoding = weights[0], 
                                   weightVectorsHSGS = weights[1], 
                                   repeat=10,
                                   bias=False, 
                                   phaseEstrategyOperator='original',
                                   testingApproaches={'hsgs'})

AVG TEST ERROR HSGS    0.32500000000000007


/opt/conda/lib/python3.7/site-packages/numpy/lib/function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/opt/conda/lib/python3.7/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [10]:
print('phase avg_error', results[0])
print('phase weights', results[2])

print('\HSGS avg_error', results[3])
print('HSGS weights', results[5])

phase avg_error nan
phase weights [0.4528429325464004, 0.3529783659443947, 0.6573994627797582, 0.3703510829880352, 0.45909297789143144, 0.7193241225090415, 0.4129918291138346, 0.9064232691643387, 0.1804516192026816, 0.7411188729132643, 0.42237404364314035, 0.42645357268494233, 0.6343798686338388, 0.5229062010283453, 0.4148859784394432, 0.0014268805627581926, 0.09226234584675208, 0.7093943937252127, 0.5243455967651972, 0.6961604635169689, 0.955468323002926, 0.6829138543754175, 0.05312869067295756, 0.30885268486379713, 0.592594687323373, 0.2351204072574643, 0.9649709995361269, 0.9450482237927939, 0.8484008808376887, 0.47232399628840227, 0.8414767148983973, 0.13111064234791392, 0.3087336572978354, 0.46299639415441707, 0.7418472006834668, 0.48582522870889666, 0.13687611879745087, 0.34353652970435833, 0.324426169672443, 0.30041890431803975]
\HSGS avg_error 0.32500000000000007
HSGS weights [67.95284293254682, 63.852978365945035, 44.15739946278012, -7.129648917011956, -11.040907022108547, 68.

## Binary Quantum Neuron Experiments 

In [8]:
output = {'model':[],
          'phase_strategy':[],
            'bias':[],
            'threshold':[],
            'lr':[],
            'avg_error':[],
            'trained_weights':[],
            'initial_weights':[],
            'neuron_outputs':[]}

In [9]:
# search space
initial_weights =  list(np.random.uniform(low=0.0, high=1.0, size=(len(X_train_bin[0]),)))
threshold_space = [0.3, 0.5, 0.7, 0.8]
lr_space = [0.02, 0.1]
bias_space = [False, True]
strategy_space = ['original']

In [ ]:
%%time
for w in initial_weights:
    for strategy in strategy_space:
        for threshold in threshold_space:
            for lr in lr_space:
                for bias in bias_space:
                    # execute experiment
                    weights = quantumNeuronFIT(X_train_bin, 
                                               y_train,
                                               init_weight=initial_weights,
                                               threshold=threshold,
                                               lrParameter=lr, 
                                               n_epochs=20, 
                                               trainingBias=bias,
                                               phaseEstrategyOperator=strategy,
                                               trainingApproaches={'hsgs'})

                    results = quantumNeuronPREDICT(X_test_bin, 
                                                   y_test,
                                                   threshold=threshold,
                                                   weightVectorsPhaseEncoding = weights[0], 
                                                   weightVectorsHSGS = weights[1], 
                                                   repeat=10,
                                                   bias=bias, 
                                                   phaseEstrategyOperator=strategy,
                                                   testingApproaches={'hsgs'})


                    output['model'].append('HSGS')
                    output['lr'].append(lr)
                    output['bias'].append(bias)
                    output['threshold'].append(threshold)
                    output['phase_strategy'].append(strategy)
                    output['avg_error'].append(results[3])
                    output['neuron_outputs'].append(results[4])
                    output['trained_weights'].append(results[5])
                    output['initial_weights'].append(w)
                    print('\n')

lrParameter:  0.02
threshold:  0.3
trainingBias:  False
phaseEstrategyOperator:  original
best error HSGS training:  124
AVG TEST ERROR HSGS    0.3125


lrParameter:  0.02
threshold:  0.3
trainingBias:  True
phaseEstrategyOperator:  original


/opt/conda/lib/python3.7/site-packages/numpy/lib/function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/opt/conda/lib/python3.7/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


best error HSGS training:  122
AVG TEST ERROR HSGS    0.45


lrParameter:  0.1
threshold:  0.3
trainingBias:  False
phaseEstrategyOperator:  original
best error HSGS training:  129
AVG TEST ERROR HSGS    0.5


lrParameter:  0.1
threshold:  0.3
trainingBias:  True
phaseEstrategyOperator:  original
best error HSGS training:  118
AVG TEST ERROR HSGS    0.4125


lrParameter:  0.02
threshold:  0.5
trainingBias:  False
phaseEstrategyOperator:  original
best error HSGS training:  118
AVG TEST ERROR HSGS    0.4


lrParameter:  0.02
threshold:  0.5
trainingBias:  True
phaseEstrategyOperator:  original
best error HSGS training:  135
AVG TEST ERROR HSGS    0.4375


lrParameter:  0.1
threshold:  0.5
trainingBias:  False
phaseEstrategyOperator:  original
best error HSGS training:  118
AVG TEST ERROR HSGS    0.4


lrParameter:  0.1
threshold:  0.5
trainingBias:  True
phaseEstrategyOperator:  original
best error HSGS training:  133
AVG TEST ERROR HSGS    0.4749999999999999


lrParameter:  0.02
thresh

In [12]:
experiment_hsgs = pd.DataFrame(output)
experiment_hsgs.to_csv('results/version4/experiment_diabetes_hsgs.csv')

## Continuously Valued Quantum Neuron Experiments

In [3]:
output = {'model':[],
          'phase_strategy':[],
            'bias':[],
            'threshold':[],
            'lr':[],
            'avg_error':[],
            'trained_weights':[],
            'initial_weights':[],
            'neuron_outputs':[]}

In [4]:
# search space
initial_weights =  list(np.random.uniform(low=0.0, high=1.0, size=(len(X_test[0]),)))
threshold_space = [0.3, 0.5, 0.7, 0.8]
lr_space = [0.02, 0.1]
bias_space = [False, True]
strategy_space = ['original', 'angle', 'radius', 'angleradius']

In [ ]:
%%time
for w in initial_weights:
    for strategy in strategy_space:
        for threshold in threshold_space:
            for lr in lr_space:
                for bias in bias_space:
                    # execute experiment
                    weights = quantumNeuronFIT(X_train, 
                                               y_train,
                                               init_weight=initial_weights,
                                               threshold=threshold,
                                               lrParameter=lr, 
                                               n_epochs=20, 
                                               trainingBias=bias,
                                               phaseEstrategyOperator=strategy,
                                               trainingApproaches={'phase-encoding'})

                    results = quantumNeuronPREDICT(X_test, 
                                                       y_test,
                                                       threshold=threshold,
                                                       weightVectorsPhaseEncoding = weights[0], 
                                                       weightVectorsHSGS = weights[1], 
                                                       repeat=10,
                                                       bias=bias, 
                                                       phaseEstrategyOperator=strategy,
                                                       testingApproaches={'phase-encoding'})

                    # get experiment results
                    output['model'].append('phase-encoding')
                    output['lr'].append(lr)
                    output['bias'].append(bias)
                    output['threshold'].append(threshold)
                    output['phase_strategy'].append(strategy)
                    output['avg_error'].append(results[0])
                    output['neuron_outputs'].append(results[1])
                    output['trained_weights'].append(results[2])
                    output['initial_weights'].append(w)
                    print('\n')

lrParameter:  0.02
threshold:  0.3
trainingBias:  False
phaseEstrategyOperator:  original

best error phase-encoding training:  196
AVG TEST ERROR PHASE   0.6875


lrParameter:  0.02
threshold:  0.3
trainingBias:  True
phaseEstrategyOperator:  original


/opt/conda/lib/python3.7/site-packages/numpy/lib/function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/opt/conda/lib/python3.7/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)



best error phase-encoding training:  115
AVG TEST ERROR PHASE   0.3125


lrParameter:  0.1
threshold:  0.3
trainingBias:  False
phaseEstrategyOperator:  original

best error phase-encoding training:  196
AVG TEST ERROR PHASE   0.6875


lrParameter:  0.1
threshold:  0.3
trainingBias:  True
phaseEstrategyOperator:  original

best error phase-encoding training:  124
AVG TEST ERROR PHASE   0.32500000000000007


lrParameter:  0.02
threshold:  0.5
trainingBias:  False
phaseEstrategyOperator:  original

best error phase-encoding training:  209
AVG TEST ERROR PHASE   0.69875


lrParameter:  0.02
threshold:  0.5
trainingBias:  True
phaseEstrategyOperator:  original


In [7]:
experiment_phase = pd.DataFrame(output)
experiment_phase.to_csv('results/version4/experiment_nonlinear_phase_vx.csv')

In [8]:
experiment_phase

,model,phase_strategy,bias,threshold,lr,avg_error,trained_weights,initial_weights,neuron_outputs
0,phase-encoding,original,False,0.3,0.02,0.68750,"[0.2613840185436901, 0.36881463029123307, 0.35...",0.076308,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,phase-encoding,original,True,0.3,0.02,0.31250,"[-0.2229383919545706, -0.30364565223549217, -0...",0.076308,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,phase-encoding,original,False,0.3,0.10,0.68750,"[0.4938983238212171, 0.7120645324859705, 0.665...",0.076308,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,phase-encoding,original,True,0.3,0.10,0.32500,"[-0.36692921341549883, -0.47345177255208143, -...",0.076308,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,phase-encoding,original,False,0.5,0.02,0.69875,"[0.24100792577640034, 0.33833772164506487, 0.3...",0.076308,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
5,phase-encoding,original,True,0.5,0.02,0.32500,"[-0.22306070519881066, -0.3036644993866432, -0...",0.076308,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
6,phase-encoding,original,False,0.5,0.10,0.68750,"[0.4937740857676634, 0.7120630788023563, 0.665...",0.076308,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
7,phase-encoding,original,True,0.5,0.10,0.32500,"[-0.36692921341549883, -0.47345177255208143, -...",0.076308,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
8,phase-encoding,original,False,0.7,0.02,0.30000,"[-0.22288195525145463, -0.300068381425296, -0....",0.076308,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
9,phase-encoding,original,True,0.7,0.02,0.32500,"[-0.22306070519881066, -0.3036644993866432, -0...",0.076308,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
